# SVM et régression.

**Comme mentionné dans la partie cours, les SVM peuvent aussi permettre d’aborder des problèmes de régression.**

On propose dans cet exercice de tester cette méthode sur le jeu de données ozone accessible sur Moodle. 
L’objectif, sur ces données, est :
>D’améliorer la prévision calculée par les services de MétéoFrance (MOCAGE) de la concentration d’ozone dans certaines stations de prélèvement à partir de cette prévision et en s’aidant d’autre variables également prévues par MétéoFrance. 

Il s’agit d’un problème dit d’adaptation statistique d’une prévision déterministe pour une amélioration locale de modèles à trop grande échelle. 

Plus précisément, deux variables peuvent être prédites :
- Soit la concentration quantitative d’ozone
- Soit le dépassement (qualitatif) d’un certain seuil fixé à 150 μg.

On s’intéressera en priorité au premier problème.

In [1]:
#Chargement des modules

import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,Normalizer

from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gbrt_minimize
import time

On charge les données et on effectue le preprocessing : 

In [2]:
df = pd.read_csv('ozone.dat', sep=' ')

#Pour reprendre les mêmes données, cependant, je n'ai pas observé d'amélioration
df['LNO'] = np.log(df['NO'])
df['LNO2'] = np.log(df['NO2'])
df['SRMH2O'] = np.sqrt(df['RMH2O'])

features = list(df.columns)
features.remove('O3obs')
features = features[0:3] + features[6:12]
encoder = LabelEncoder().fit(df['STATION'])
df['STATION'] = encoder.transform(df['STATION'])

X = df[features]
y = df['O3obs']

Nos X et y sont prêts pour entraîner des modèles, on va en faire plusieurs.

Modèle avec les paramètres de base :

In [3]:
reg1 = SVR()
t = time.time()
score_reg = -np.mean(
    cross_val_score(
        reg1, X, y, cv=5, n_jobs=-1, scoring="neg_mean_squared_error"))
print("Score moyen:{x} \nTemps : {tps}".format(x=score_reg,
                                               tps=time.time() - t))

Score moyen:1170.672710453135 
Temps : 0.8715059757232666


En cherchant les paramètres optimaux avec scikit-optimize :

In [4]:
reg_sk = SVR()
space = [
    Real(1, 10**3, prior='log-uniform', name='C'),
    Real(10**-10, 10**-4, prior='log-uniform', name='gamma'),
    Real(10**-8, 1, prior='log-uniform', name='epsilon')
]


@use_named_args(space)
def objective_rbf(**params):
    reg_sk.set_params(**params)
    return -np.mean(
        cross_val_score(
            reg_sk, X, y, cv=5, n_jobs=3, scoring="neg_mean_squared_error"))


t = time.time()
res_skopt = gbrt_minimize(objective_rbf, space, n_calls=80)
print("Score moyen:{x} \nTemps : {tps}".format(x=res_skopt.fun,
                                               tps=time.time() - t))

Score moyen:873.2821978295767 
Temps : 16.16158890724182


Avec Hyperopt, on peut faire des choses très intéressante aussi, il faut cependant du temps et de la puissance de calcul :

In [5]:
from hpsklearn import HyperoptEstimator, svr, any_preprocessing
from hyperopt import tpe, rand
import numpy as np


def f_perte(y_target, y_pred):
    return np.sum((y_target - y_pred)**2) / len(y_pred)


estim = HyperoptEstimator(regressor=svr('MySvr'),
                          preprocessing=any_preprocessing('prepro'),
                          algo=tpe.suggest,
                          loss_fn=f_perte,
                          max_evals=200,
                          trial_timeout=1,
                          n_jobs=-1)

estim.fit(X,
          y,
          n_folds=5,
          cv_shuffle=False,
          random_state=None)

print(estim.best_model())

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 20.71trial/s, best loss: 940.3866244772781]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:00<00:00,  1.72trial/s, best loss: 879.4708494903123]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:00<00:00, 11.48trial/s, best loss: 873.326233867758]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135/135 [00:01<00:00,  1.05s/trial, best loss: 873.326233867758]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169/169 [00:00<00:00,  4.99trial/s, best loss: 872.5475254374381]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00,  3.90trial/s, best loss: 872.129292638765]
{'learner': SVR(C=0.03307304999211984, cache_size=512, degree=1,
    epsilon=0.28520513716618456, gamma='auto', kernel='linear',
    max_iter=575120308.0, tol=2.666776135510756e-05), 'preprocs': (), 'ex_preprocs': ()}


On obtient ce résultat pour 20000 itérations :

{'learner': SVR(C=99452.83389848414, cache_size=512, degree=1, epsilon=39.81008532761599,
    gamma=0.003122060619305805, max_iter=154856981.0, tol=0.005409093264476527), 'preprocs': (PCA(n_components=4, whiten=True),), 'ex_preprocs': ()}
 
Si l'on veux un modèle plus "propre" on peux définir un pipeline, voici un exemple :

In [63]:
from sklearn.pipeline import make_pipeline

regressor = make_pipeline(
    PCA(n_components=4, whiten=True),
    SVR(C=99452.83389848414, cache_size=512, degree=1, epsilon=39.81008532761599,
    gamma=0.003122060619305805, max_iter=154856981.0, tol=0.005409093264476527))
regressor

Pipeline(steps=[('pca', PCA(n_components=4, whiten=True)),
                ('svr',
                 SVR(C=99452.83389848414, cache_size=512, degree=1,
                     epsilon=39.81008532761599, gamma=0.003122060619305805,
                     max_iter=154856981.0, tol=0.005409093264476527))])

In [67]:
score_regressor = -np.mean(
    cross_val_score(regressor,
                    X,
                    y,
                    cv=5,
                    n_jobs=-1,
                    scoring="neg_mean_squared_error"))
score_regressor

785.1792220067052

Intéressons nous au deuxième problème:

In [110]:
#On crée un vecteur de réponse adapté au problème
from hpsklearn import svc
y_class=[0 if i==-1 else 1 for i in [np.sign(i-150) for i in y]]

#On garde la fonction de perte de base étant 1 -accuracy

estim = HyperoptEstimator(classifier=svc('MySvr'),
                          preprocessing=any_preprocessing('prepro'),
                          algo=rand.suggest,
                          max_evals=300,
                          trial_timeout=2,
                          n_jobs=-1)
estim.fit(X,y_class,
          n_folds=5,
          cv_shuffle=False,
          random_state=None)

print(estim.best_model())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 12.10trial/s, best loss: 0.13352545629202694]
{'learner': SVC(C=2.6533169056501866, cache_size=512, coef0=0.04478454019832232, degree=1,
    gamma=0.0374703715666424, kernel='sigmoid', max_iter=25177894.0,
    random_state=2, tol=5.171799415457331e-05), 'preprocs': (StandardScaler(),), 'ex_preprocs': ()}


Résultat pour 200 itération:

{'learner': SVC(C=2.6533169056501866, cache_size=512, coef0=0.04478454019832232, degree=1,
    gamma=0.0374703715666424, kernel='sigmoid', max_iter=25177894.0,
    random_state=2, tol=5.171799415457331e-05), 'preprocs': (StandardScaler(),), 'ex_preprocs': ()}

In [111]:
clf=make_pipeline(
    StandardScaler(),
    SVC(C=2.6533169056501866, cache_size=512, coef0=0.04478454019832232, degree=1,
    gamma=0.0374703715666424, kernel='sigmoid', max_iter=25177894.0,
    random_state=2, tol=5.171799415457331e-05))

score_clf = np.mean(
    cross_val_score(clf,
                    X,
                    y_class,
                    cv=5,
                    n_jobs=-1,
                    scoring="accuracy"))

In [112]:
score_clf

0.8664703717335296